In [9]:
from ultralytics import YOLO
import os
import yaml

In [15]:
with open("../plantDoc-4/data.yaml", "r") as f:
    data_cfg = yaml.safe_load(f)

# Extract class names from YAML
original_class_names = data_cfg["names"]

In [17]:
crops = set()

# List for storing cleaned class names
cleaned_class_names = []

# Process each class name
for name in original_class_names:
    # Normalize: replace underscores with space and lowercase
    cleaned = name.replace("_", " ").lower()
    words = cleaned.split()

    # Identify and normalize crop name
    if words[0] == "bell" and len(words) > 1 and words[1] == "pepper":
        crop = "Bell pepper"
        rest = " ".join(words[2:])
    elif words[0] in ["soyabean", "soybean"]:
        crop = "Soybean"
        rest = " ".join(words[1:])
    elif words[0] == "grape":
        crop = "Grape"
        rest = " ".join(words[1:])
    else:
        crop = words[0].capitalize()
        rest = " ".join(words[1:])

    # Add crop to set
    crops.add(crop)

    # Reconstruct cleaned class name
    if rest:
        cleaned_name = f"{crop} {rest}"
    else:
        cleaned_name = crop

    cleaned_class_names.append(cleaned_name)

In [20]:
# Print cleaned class names
print("All Class Names:")
for name in cleaned_class_names:
    print(f"-- {name}")

All Class Names:
-- Apple scab leaf
-- Apple leaf
-- Apple rust leaf
-- Bell pepper leaf spot
-- Bell pepper leaf
-- Blueberry leaf
-- Cherry leaf
-- Corn gray leaf spot
-- Corn leaf blight
-- Corn rust leaf
-- Peach leaf
-- Potato leaf early blight
-- Potato leaf late blight
-- Potato leaf
-- Raspberry leaf
-- Soybean leaf
-- Soybean leaf
-- Squash powdery mildew leaf
-- Strawberry leaf
-- Tomato early blight leaf
-- Tomato septoria leaf spot
-- Tomato leaf bacterial spot
-- Tomato leaf late blight
-- Tomato leaf mosaic virus
-- Tomato leaf yellow virus
-- Tomato leaf
-- Tomato mold leaf
-- Tomato two spotted spider mites leaf
-- Grape leaf black rot
-- Grape leaf


In [21]:
# Print unique crop names
print("\nUnique Crop Names:")
for crop in sorted(crops):
    print(f"-- {crop}")


Unique Crop Names:
-- Apple
-- Bell pepper
-- Blueberry
-- Cherry
-- Corn
-- Grape
-- Peach
-- Potato
-- Raspberry
-- Soybean
-- Squash
-- Strawberry
-- Tomato


In [25]:
# --- Global constants ---
DATA_YAML = "../plantDoc-4/data.yaml"
IMGSZ = 640
EPOCHS = 25
BATCH_SIZE = 16
RESULTS_DIR = "../runs/sota"
os.makedirs(RESULTS_DIR, exist_ok=True)

In [5]:
def train_model(model_path):
    model_name = os.path.splitext(os.path.basename(model_path))[0]
    save_dir = os.path.join(RESULTS_DIR, model_name)
    
    print(f"\nTraining model: {model_name}")
    print(f"Data yaml: {DATA_YAML}")
    print(f"Image size: {IMGSZ}")
    print(f"Epochs: {EPOCHS}")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Results will be saved to: {save_dir}")
    
    model = YOLO(model_path)
    model.train(
        data=DATA_YAML,
        imgsz=IMGSZ,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        project=RESULTS_DIR,
        name=model_name,
        exist_ok=True,
    )
    print(f"Training completed for {model_name}!")

In [6]:
train_model("../sota/yolov8n.pt")


Training model: yolov8n
Data yaml: ../plantDoc-4/data.yaml
Image size: 640
Epochs: 25
Batch size: 16
Results will be saved to: ../runs/sota\yolov8n
Ultralytics 8.3.214  Python-3.10.18 torch-2.9.0+cpu CPU (13th Gen Intel Core(TM) i5-1335U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../plantDoc-4/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=../sota/yolov8n.pt, momentum=0.937, mosaic=1.0, mult

In [28]:
MODEL_PATH = "../runs/sota/yolov8n/weights/last.pt"  # checkpoint from 25 epochs

In [29]:
# Load model from previous run
model = YOLO(MODEL_PATH)  # This will automatically resume from last.pt
model.info()

Model summary: 129 layers, 3,016,698 parameters, 0 gradients, 8.2 GFLOPs


(129, 3016698, 0, 8.2253312)

In [31]:
# Train with new settings
model.train(
    data=DATA_YAML,
    imgsz=IMGSZ,
    epochs=100,         # Training will resume and continue to 100 epochs
    batch=BATCH_SIZE,
    project=RESULTS_DIR,
    name="yolov8n",
    optimizer="Adam",      # Changed optimizer
    lr0=0.001,             # Learning rate
    cos_lr=True,           # Cosine learning rate scheduler
    patience=10,           # For early stopping
    exist_ok=True,
    verbose=True
)

New https://pypi.org/project/ultralytics/8.3.217 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.214  Python-3.10.18 torch-2.9.0+cpu CPU (13th Gen Intel Core(TM) i5-1335U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=../plantDoc-4/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=../runs/sota/yolov8n/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  8,  9, 10, 14, 15, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 28])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000014DA6BD0D30>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([0.        , 0.001001  , 0.002002  , 0.003003  , 0.004004  ,
       0.00500501, 0.00600601, 0.00700701, 0.00800801, 0.00900901,
       0.01001001, 0.01101101, 0.01201201, 0.01301301, 0.01401401,
       0.01501502, 0.01601602, 0.01701702, 0.01801802, 0.01901902,
       0.02002002, 0.02102102, 0.02202202, 0.02302302, 0.02402402,
       0.02502503, 0.02602603, 0.02702703, 0.02802803, 0.02902903,
       0.03003003, 0.03103103, 0.03203203, 0.03303303, 0.03403403,
       0.03503504, 0.03603604, 0.03703704, 0.03803804, 0.03903904,
       0.04004004, 0.0

In [35]:
# Load the trained model (best.pt or last.pt depending on your preference)
model = YOLO('../runs/sota/yolov8n/weights/best.pt')  # or 'last.pt'

In [ ]:
metrics = model.val(
    data=DATA_YAML,
    imgsz=IMGSZ,
    split='test',
    batch=BATCH_SIZE,
    project="../runs/sota",  # Base directory
    name="yolov8n/val",      # Subdirectory inside project
    exist_ok=True,
    verbose=True
)

Ultralytics 8.3.214  Python-3.10.18 torch-2.9.0+cpu CPU (13th Gen Intel Core(TM) i5-1335U)
val: Fast image access  (ping: 0.20.1 ms, read: 127.898.0 MB/s, size: 172.1 KB)
val: Scanning C:\Users\ADITHYA\OneDrive\Desktop\YOLO-Tweaks\PlantDoc-4\test\labels.cache... 246 images, 1 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 246/246  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 16/16 0.6it/s 26.3s1.6s
                   all        246        484      0.582        0.6      0.606       0.45
       Apple Scab Leaf         10         13      0.887      0.606       0.68      0.481
            Apple leaf          9         10      0.409        0.9      0.606      0.489
       Apple rust leaf         11         12      0.726      0.884      0.675      0.477
 Bell_pepper leaf spot          8         11      0.309      0.636      0.524      0.404
      Bell_pepper leaf          9         15      0.388      0.267        0.3       0.22
 

In [36]:
results = model.predict(
    source='../plantDoc-4/test/images',
    imgsz=IMGSZ,
    save=True,
    project='../runs/sota',
    name='yolov8n/predict',
    exist_ok=True
)


image 1/246 c:\Users\ADITHYA\OneDrive\Desktop\YOLO-Tweaks\code\..\plantDoc-4\test\images\-2320-Bacterial-Spot-and-Speck_jpg.rf.9d63c6e88cb474baa4d4eab1681f6d41.jpg: 480x640 3 Tomato Septoria leaf spots, 153.3ms
image 2/246 c:\Users\ADITHYA\OneDrive\Desktop\YOLO-Tweaks\code\..\plantDoc-4\test\images\0000_jpg.rf.d9be28abba2b4e67431ed005bdde14de.jpg: 640x640 1 grape leaf black rot, 269.7ms
image 3/246 c:\Users\ADITHYA\OneDrive\Desktop\YOLO-Tweaks\code\..\plantDoc-4\test\images\000_jpg.rf.e32ba3f04d1e855e58b1b3f5475cf1e5.jpg: 480x640 1 Bell_pepper leaf, 106.9ms
image 4/246 c:\Users\ADITHYA\OneDrive\Desktop\YOLO-Tweaks\code\..\plantDoc-4\test\images\00_jpg.rf.567521ea9c2d0153cb8b96c5bd18e692.jpg: 480x640 1 Bell_pepper leaf, 1 Tomato Septoria leaf spot, 134.4ms
image 5/246 c:\Users\ADITHYA\OneDrive\Desktop\YOLO-Tweaks\code\..\plantDoc-4\test\images\00pe_jpg.rf.a17680fc7cdfa4c78c70f3ef1a81fd79.jpg: 640x480 (no detections), 148.0ms
image 6/246 c:\Users\ADITHYA\OneDrive\Desktop\YOLO-Tweaks\cod